# Porownanie działania, usuwanie szumów oraz IDF

In [2]:
import numpy as np
from numpy.core.numeric import outer

#TODO
#Less articles, memory error at termbydocument
def readDataSet(filePath, numOfArticles):

    with open(filePath, "r") as file:
        content = file.read()

        print("Replacing special characters")
        content = content.replace(',', '').replace('.', '').replace(':', '').replace('"', '').replace('-', ' ').replace('?', '').replace('!', '').replace('(', '').replace(')', '').replace('\'s', '').lower()

        #Getting only limited amount of documents
        content = content.split('\n\n')
        content = content[:numOfArticles]
        content = "\n\n".join(content)

        bagOfWords = {}
        articles = {}

        # Get the words counter
        index = 0
        words = content.split()

        for word in words:

            bagOfWords.setdefault(word, {
                "counter": 0,
                "index": 0
            })

            bagOfWords[word]["counter"] += 1
            if bagOfWords[word]["counter"] == 1:
                bagOfWords[word]["index"] = index
                index += 1

        # Getting separate articles
        index = 0
        content = content.split('\n\n')

        for article in content:
            
            article = article.split('\n')

            # Getting the title and removing it from article data
            title = article[0]
            article = "\n".join(article[1:])

            if title not in articles:
                articles[title] = {
                    "article": article,
                    "index": index
                }
                index += 1
            
    return articles, bagOfWords
            
def getTermByDocumentMatrix(bagOfWords, articles):
    result = np.zeros((len(bagOfWords), len(articles)))

    for key, value in articles.items():
        words = value['article'].split()
        words.extend(key.split())

        for word in words:
            result[bagOfWords[word]["index"]][value['index']] += 1
    
    return result

def IDF(articles, termByDocument):
    for wordIndex in range(len(termByDocument)):
        counter = 0
        for docIndex in range(len(termByDocument[wordIndex])):
            if termByDocument[wordIndex][docIndex] > 0:
                counter += 1
    
        idf = np.log(len(articles)/counter)
        termByDocument[wordIndex,:] *= idf

def search(words, termByDocument, bagOfWords, k):
    q = np.zeros((len(termByDocument)))
    words = words.split()

    for word in words:
        if word in bagOfWords:
            q[bagOfWords[word]['index']] += 1
    
    q /= np.linalg.norm(q)

    result = []
    for docIndex in range(len(termByDocument[0])):
        column = termByDocument[:,docIndex]
        result.append(q.dot(column/np.linalg.norm(column)))
    
    result = [[index, value] for index, value in enumerate(result)]
    result.sort(key = lambda x: x[1], reverse=True)

    return np.array(result[:k % len(termByDocument[0])])

def getResultArticles(results, articles):
    result = ''
    temp = list(articles.items())
    for r in results:
        for title, data in temp:
            if data['index'] == r[0]:
                result += title + '\n'
                result += data['article'] + '\n\n'
                break
            
    return result

def noiseReducion(termByDocument, k):
    U, S, V = np.linalg.svd(termByDocument, full_matrices=False)
    termByDocument = np.zeros((len(U), len(V)))
    for i in range(k):
        termByDocument += S[i] * np.outer(U.T[i], V[i])
    
    return termByDocument

## Porownanie wyników z użyciem redukcji szumów

In [6]:
k = 100
articles, bagOfWords = readDataSet('corpus.txt', 1000)
termByDocument = getTermByDocumentMatrix(bagOfWords, articles)
IDF(articles, termByDocument)
approximatedTermByDocument = noiseReducion(termByDocument, k)

result = search("mathematics", termByDocument, bagOfWords, 3)
print(getResultArticles(result, articles))

print('================================')
print()

result = search("mathematics", approximatedTermByDocument, bagOfWords, 3)
print(getResultArticles(result, articles))

Replacing special characters
mathematics
mathematics is the study of numbers patterns and shapes people who are experts at mathematics are called mathematicians
mathematics can find useful answers to real problems a common saying in the us is do the math meaning check the answer and see that it is right for example if we want to know if a new medicine will help to cure a disease mathematics is used to answer that question
the way mathematicians solve problems is deduction deduction is the use of thinking to discover new ideas from old for example from the simple idea of a triangle mathematicians found the pythagorean theorem that in a right triangle the sum of the squares of the two shorter sides is equal to the square of the longest side thus if we know that the shorter sides of a right triangle are 3 inches and 4 inches we can deduce that the longest side is 5 inches because the square of 33^2 is 9 the square of 44^2 is 16 and the square of 5 5^2 is 25; and 9 + 16 = 25
mathematics is

In [7]:
k = 200
articles, bagOfWords = readDataSet('corpus.txt', 1000)
termByDocument = getTermByDocumentMatrix(bagOfWords, articles)
IDF(articles, termByDocument)
approximatedTermByDocument = noiseReducion(termByDocument, k)

result = search("mathematics", termByDocument, bagOfWords, 3)
print(getResultArticles(result, articles))

print('================================')
print()

result = search("mathematics", approximatedTermByDocument, bagOfWords, 3)
print(getResultArticles(result, articles))

Replacing special characters
mathematics
mathematics is the study of numbers patterns and shapes people who are experts at mathematics are called mathematicians
mathematics can find useful answers to real problems a common saying in the us is do the math meaning check the answer and see that it is right for example if we want to know if a new medicine will help to cure a disease mathematics is used to answer that question
the way mathematicians solve problems is deduction deduction is the use of thinking to discover new ideas from old for example from the simple idea of a triangle mathematicians found the pythagorean theorem that in a right triangle the sum of the squares of the two shorter sides is equal to the square of the longest side thus if we know that the shorter sides of a right triangle are 3 inches and 4 inches we can deduce that the longest side is 5 inches because the square of 33^2 is 9 the square of 44^2 is 16 and the square of 5 5^2 is 25; and 9 + 16 = 25
mathematics is

In [8]:
k = 300
articles, bagOfWords = readDataSet('corpus.txt', 1000)
termByDocument = getTermByDocumentMatrix(bagOfWords, articles)
IDF(articles, termByDocument)
approximatedTermByDocument = noiseReducion(termByDocument, k)

result = search("mathematics", termByDocument, bagOfWords, 3)
print(getResultArticles(result, articles))

print('================================')
print()

result = search("mathematics", approximatedTermByDocument, bagOfWords, 3)
print(getResultArticles(result, articles))

Replacing special characters
mathematics
mathematics is the study of numbers patterns and shapes people who are experts at mathematics are called mathematicians
mathematics can find useful answers to real problems a common saying in the us is do the math meaning check the answer and see that it is right for example if we want to know if a new medicine will help to cure a disease mathematics is used to answer that question
the way mathematicians solve problems is deduction deduction is the use of thinking to discover new ideas from old for example from the simple idea of a triangle mathematicians found the pythagorean theorem that in a right triangle the sum of the squares of the two shorter sides is equal to the square of the longest side thus if we know that the shorter sides of a right triangle are 3 inches and 4 inches we can deduce that the longest side is 5 inches because the square of 33^2 is 9 the square of 44^2 is 16 and the square of 5 5^2 is 25; and 9 + 16 = 25
mathematics is

## Wnioski:

Jak możemy zauważyć, najlepsza jest wartość 300 dla parametru "k"

## Porownanie wyników z użyciem IDF

In [10]:
articles, bagOfWords = readDataSet('corpus.txt', 1000)
termByDocument = getTermByDocumentMatrix(bagOfWords, articles)
termByDocument2 = getTermByDocumentMatrix(bagOfWords, articles)
IDF(articles, termByDocument)

result = search("mathematics", termByDocument, bagOfWords, 5)
print(getResultArticles(result, articles))

print('================================')
print()

result = search("mathematics", termByDocument2, bagOfWords, 5)
print(getResultArticles(result, articles))

Replacing special characters
mathematics
mathematics is the study of numbers patterns and shapes people who are experts at mathematics are called mathematicians
mathematics can find useful answers to real problems a common saying in the us is do the math meaning check the answer and see that it is right for example if we want to know if a new medicine will help to cure a disease mathematics is used to answer that question
the way mathematicians solve problems is deduction deduction is the use of thinking to discover new ideas from old for example from the simple idea of a triangle mathematicians found the pythagorean theorem that in a right triangle the sum of the squares of the two shorter sides is equal to the square of the longest side thus if we know that the shorter sides of a right triangle are 3 inches and 4 inches we can deduce that the longest side is 5 inches because the square of 33^2 is 9 the square of 44^2 is 16 and the square of 5 5^2 is 25; and 9 + 16 = 25
mathematics is

## Wnioski

Wyniki lekko się różnią ale są bardzo podobnie celne
